<a href="https://colab.research.google.com/github/waytoakash/waytoakash/blob/main/CRM_tracker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Update CRM data for CRM [dashboard](https://datastudio.google.com/u/0/reporting/03ddd963-67b4-4857-9105-76b512a687e4/page/p_miiln3o0qc
) and [backend sheet](https://docs.google.com/spreadsheets/d/1sqPSxZAts7fZpB5TyiJf7OY_HDkyuB79qKK0vYD4aMg/edit#gid=257777437)

Author : Angshuman M

Date : 21 January 2022

Ver : 1.0


---

#Authentication

In [ ]:

# Google Drive
from google.colab import drive
drive.mount('/content/gdrive')


# Google Sheets
from google.colab import auth
auth.authenticate_user()

import pandas as pd
import numpy as np
import os
import sys

from datetime import datetime
import dateutil
from dateutil.parser import parse
import re
from functools import reduce

# import gspread
# from oauth2client.client import GoogleCredentials
# gc = gspread.authorize(GoogleCredentials.get_application_default())

import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)
from gspread_dataframe import set_with_dataframe


folder_date = input("Enter Folder Date :")
folder_date = folder_date
m1 = folder_date[3:6]
y1 = folder_date[-4:]
date_folder = m1+" "+y1
file_path = "/content/gdrive/MyDrive/Insights_Datalake/Datewise Dumps - Aug Onwards/"+date_folder+"/"+folder_date+"/"

file_path

Mounted at /content/gdrive
Enter Folder Date :09 November 2022


'/content/gdrive/MyDrive/Insights_Datalake/Datewise Dumps - Aug Onwards/Nov 2022/09 November 2022/'

#custom functions

In [ ]:
def changeDateFormat(row):
  return pd.to_datetime(row, errors='coerce').dt.date
def changeDateTimeFormat(row):
  return pd.to_datetime(row, errors='coerce').dt.time

def getDump(mastermerge, dump_name,start_date, end_date):
  start_date = pd.to_datetime(start_date).date()
  end_date = pd.to_datetime(end_date).date()
  file_path = "/content/gdrive/MyDrive/Insights_Datalake/Datewise Dumps - Aug Onwards/"
  reg = re.compile(".+" + dump_name+".csv")


  for (root, dirs, files) in os.walk(file_path):
      for file in files:
        path = os.path.join(root, file)
        date = str(path).split('/')[7:][0]
        date = pd.to_datetime(date).date()

        if bool(re.match(reg, path)) & (date >= start_date) & (date <= end_date):
          temp = pd.read_csv(path,parse_dates=True)
          temp['DumpDate'] = date
          read_dump_name = str(path).split('/')[9:][0]
          print(read_dump_name + " on Date:: " + str(date) +  " Dump of shape:: " + str(temp.shape) + " read from path:: "+ path)
          mastermerge = mastermerge.append(temp)

  mastermerge[['DumpDate']] = mastermerge[['DumpDate']].apply(changeDateFormat)
  return mastermerge

#get data from spreadsheetID and Range Name function
def get_data(spreadSheetId, rangeName):
    wb = gc.open_by_key(spreadSheetId)
    ws = wb.worksheet(rangeName)
    rows = ws.get_all_values()
    return rows

#get all data in a sheet as df and there 1st row things as the df labels.
#If sheets don't have their labels in the first row than we have to do it manuaaly as per the data
def get_data_df(spreadSheetId, rangeName):
  l = get_data(spreadSheetId, rangeName)
  ldf = pd.DataFrame.from_records(l[1:], columns = l[0])
  return ldf

#read Data :Central Lead Gen Meeting Fix Audit

In [ ]:
key = "1iKgiRR-lOQ-zM6v6bh_1P3vkIujVCeBzVVr_nHuoDMU"
range = "Society Master"
master_societies = get_data_df(key,range)
master_societies = master_societies.loc[master_societies['societyStatus']=="ACTIVE"]
master_societies = master_societies.rename(columns={'Kibana ID':'KibanaID'})

In [ ]:
forum = pd.read_csv(file_path+"Sai/Societywise Forum Usage.csv")

In [ ]:

notice = pd.read_csv(file_path+"Sai/Notice Board Usage.csv")
complaints = pd.read_csv(file_path+"Sai/Complaint created last 7 days.csv")

In [ ]:

complaints30 = pd.read_csv(file_path+"Sai/Complaints created last 30 days.csv")
homechef7 = pd.read_csv(file_path+"Hari/Home-Chef-Orders7.csv")
homechef30 = pd.read_csv(file_path+"Hari/Home-Chef-Orders30.csv")
sos7 = pd.read_csv(file_path+"Hari/SOS Alerts Raised Last 7 days.csv")
sos30 = pd.read_csv(file_path+"Hari/SOS Alerts Raised Last 30 days.csv")
walkytalky = pd.read_csv(file_path+"Hari/Walky Talky (Last 7 30 days).csv")
gp = pd.read_csv(file_path+"Hari/Guard Patrolling (last 7 30 days).csv")
latest_complaints = pd.read_csv(file_path+"Hari/last complaint date societywise.csv")
latest_forums = pd.read_csv(file_path+"Hari/latest forum date societywise.csv")
latest_notice = pd.read_csv(file_path+"Hari/latest notice date societywise.csv")
society_staff_count = pd.read_csv(file_path+"Sai/Societywise count of Staff.csv")

#clean Data

In [ ]:
forum1 = forum.groupby('Society ID').sum().reset_index()
forum1 = forum1.rename(columns={'Society ID':'KibanaID'})
notice1 = notice.groupby('Society ID').sum().reset_index()
notice1 = notice.rename(columns={'Society ID':'KibanaID'})
complaints['Resolved'] = complaints['Count'] - complaints['Not Resolved']
complaints1 = complaints.groupby('party → id').sum().reset_index()
complaints1 = complaints1.rename(columns={'party → id':"KibanaID"})
complaints1['Count'] = pd.to_numeric(complaints1['Count'])
x = sum(complaints1['Resolved'])/sum(complaints1['Count'])
a1= str(round(x*100,2))
complaints30['Resolved'] = complaints30['Count'] - complaints30['Not Resolved']
complaints2 = complaints30.groupby('party → id').sum().reset_index()
complaints2 = complaints2.rename(columns={'party → id':"KibanaID"})
complaints2['Count'] = pd.to_numeric(complaints2['Count'])
x = sum(complaints2['Resolved'])/sum(complaints2['Count'])
a2= str(round(x*100,2))
walkytalky = walkytalky.rename(columns={"Society ID":"KibanaID"})
gp = gp.rename(columns={"Society ID":"KibanaID"})
sos7 = sos7.rename(columns={"society_id":"KibanaID"})
sos30 = sos30.rename(columns={"society_id":"KibanaID"})
homechef7 = homechef7.rename(columns={"chef_info_society_id":"KibanaID"})
homechef30 = homechef30.rename(columns={"chef_info_society_id":"KibanaID"})

latest_complaints = latest_complaints.rename(columns={"Societ ID":"KibanaID"})
latest_forums = latest_forums.rename(columns={"Society ID":"KibanaID"})
latest_notice = latest_notice.rename(columns={"Society ID":"KibanaID"})

society_staff_count = society_staff_count.groupby("Society ID").agg({"Staff Count":"sum"}).reset_index()
society_staff_count.rename(columns={"Society ID":"KibanaID"},inplace="True")
society_staff_count

'''
latest_complaints['Last ticket Date'] = str(latest_complaints['Last ticket Date'])
latest_complaints['Last ticket Date'].str.split("T",expand=True)
latest_complaints['Last ticket Date'] = pd.to_datetime(latest_complaints['Last ticket Date']).dt.date()


latest_forums['Last Forum Date'] = latest_forums['Last Forum Date'].str.split("T",expand=True)
latest_forums['Last Forum Date'] = pd.to_datetime(latest_forums['Last Forum Date']).dt.date()

latest_notice['Lastest Notice'] = pd.to_datetime(latest_notice['Lastest Notice']).dt.date()
'''

'\nlatest_complaints[\'Last ticket Date\'] = str(latest_complaints[\'Last ticket Date\'])\nlatest_complaints[\'Last ticket Date\'].str.split("T",expand=True)\nlatest_complaints[\'Last ticket Date\'] = pd.to_datetime(latest_complaints[\'Last ticket Date\']).dt.date()\n\n\nlatest_forums[\'Last Forum Date\'] = latest_forums[\'Last Forum Date\'].str.split("T",expand=True)\nlatest_forums[\'Last Forum Date\'] = pd.to_datetime(latest_forums[\'Last Forum Date\']).dt.date()\n\nlatest_notice[\'Lastest Notice\'] = pd.to_datetime(latest_notice[\'Lastest Notice\']).dt.date()\n'

In [ ]:
societies  = master_societies[['City','KibanaID','Original Closure Number','Society Name','societyStatus','Live Product','Latest Signing Date','Sold Flats','Downloads',
                               'RD','KAM Usage', 'Sr.KAM Usage', 'FSD Data', 'FSD Expert','FSD Usage', 'Sr. FSD Expert']]


#calculate data

In [ ]:
staff_count = pd.merge(societies,society_staff_count,how="left",on="KibanaID")
staff_count = staff_count.fillna(0)

In [ ]:
forum_usg = pd.merge(societies,forum1,how="left",on="KibanaID")

In [ ]:
notice_usg = pd.merge(societies,notice1,how="left",on="KibanaID")

In [ ]:
complaint_usg_7 = pd.merge(societies,complaints1,how="left",on="KibanaID")

In [ ]:
complaint_usg_30 = pd.merge(societies,complaints2,how="left",on="KibanaID")

In [ ]:
walky_talky_usg = pd.merge(societies,walkytalky,how="left",on="KibanaID")

In [ ]:

gp_usg = pd.merge(societies,gp,how="left",on="KibanaID")

In [ ]:
sos_usg_7 = pd.merge(societies,sos7,how="left",on="KibanaID")
sos_usg_30 = pd.merge(societies,sos30,how="left",on="KibanaID")

In [ ]:
homechef_usg_7 = pd.merge(societies,homechef7,how="left",on="KibanaID")
homechef_usg_30 = pd.merge(societies,homechef30,how="left",on="KibanaID")

In [ ]:
latest_comp = pd.merge(societies,latest_complaints,how="left",on="KibanaID")

In [ ]:
latest_noti = pd.merge(societies,latest_notice,how="left",on="KibanaID")

In [ ]:
latest_foru = pd.merge(societies,latest_forums,how="left",on="KibanaID")


#write to sheets

In [ ]:
#output sheet - https://docs.google.com/spreadsheets/d/1sqPSxZAts7fZpB5TyiJf7OY_HDkyuB79qKK0vYD4aMg/edit#gid=0

#forum_usage
spreadsheetid = "1sqPSxZAts7fZpB5TyiJf7OY_HDkyuB79qKK0vYD4aMg"
range = "forum"
wb = gc.open_by_key(spreadsheetid)
ws = wb.worksheet(range)
df = pd.DataFrame(forum_usg)
set_with_dataframe(ws, df)

In [ ]:

#notice_usage
range = "notice"
wb = gc.open_by_key(spreadsheetid)
ws = wb.worksheet(range)
df = pd.DataFrame(notice_usg)
set_with_dataframe(ws, df)


#complaints_usage
range = "complaints_7"
wb = gc.open_by_key(spreadsheetid)
ws = wb.worksheet(range)
df = pd.DataFrame(complaint_usg_7)
set_with_dataframe(ws, df)


range = "complaints_30"
wb = gc.open_by_key(spreadsheetid)
ws = wb.worksheet(range)
df = pd.DataFrame(complaint_usg_30)
set_with_dataframe(ws, df)

#walky
range = "wt"
wb = gc.open_by_key(spreadsheetid)
ws = wb.worksheet(range)
df = pd.DataFrame(walky_talky_usg)
set_with_dataframe(ws, df)

#SOS
range = "sos7"
wb = gc.open_by_key(spreadsheetid)
ws = wb.worksheet(range)
df = pd.DataFrame(sos_usg_7)
set_with_dataframe(ws, df)

range = "sos30"
wb = gc.open_by_key(spreadsheetid)
ws = wb.worksheet(range)
df = pd.DataFrame(sos_usg_30)
set_with_dataframe(ws, df)

#homechef
range = "homechef7"
wb = gc.open_by_key(spreadsheetid)
ws = wb.worksheet(range)
df = pd.DataFrame(homechef_usg_7)
set_with_dataframe(ws, df)

range = "homechef30"
wb = gc.open_by_key(spreadsheetid)
ws = wb.worksheet(range)
df = pd.DataFrame(homechef_usg_30)
set_with_dataframe(ws, df)

#latest_notice
range = "latest_notice"
wb = gc.open_by_key(spreadsheetid)
ws = wb.worksheet(range)
df = pd.DataFrame(latest_noti)
set_with_dataframe(ws, df)

#latest_notice
range = "latest_forum"
wb = gc.open_by_key(spreadsheetid)
ws = wb.worksheet(range)
df = pd.DataFrame(latest_foru)
set_with_dataframe(ws, df)

#latest_notice
range = "latest_complaints"
wb = gc.open_by_key(spreadsheetid)
ws = wb.worksheet(range)
df = pd.DataFrame(latest_comp)
set_with_dataframe(ws, df)

#society_staff
range = "staff_count"
wb = gc.open_by_key(spreadsheetid)
ws = wb.worksheet(range)
df = pd.DataFrame(staff_count)
set_with_dataframe(ws, df)

#deactivated societies

In [ ]:
key = "1sqPSxZAts7fZpB5TyiJf7OY_HDkyuB79qKK0vYD4aMg"
range = "Master-deactivated"
deactive_societies = get_data_df(key,range)
new_header = deactive_societies.iloc[0]

In [ ]:
 #grab the first row for the header
deactive = deactive_societies[1:] #take the data less the header row
deactive.columns = new_header # set the

deactive_helpdesk = deactive.iloc[:,[0,1,2,3,4]]
deactive_helpdesk.rename(columns={"society_id":"KibanaID"},inplace=True)
deactive_helpdesk = pd.merge(deactive_helpdesk,societies,how="left",on="KibanaID")
deactive_forums = deactive.iloc[:,[6,7,8,9,10]]
deactive_forums.rename(columns={"society_id":"KibanaID"},inplace=True)
deactive_forums = pd.merge(deactive_forums,societies,how="left",on="KibanaID")

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
#complaints_deactive_societies

spreadsheetid = "1sqPSxZAts7fZpB5TyiJf7OY_HDkyuB79qKK0vYD4aMg"
range = "deactive_helpdesk"
wb = gc.open_by_key(spreadsheetid)
ws = wb.worksheet(range)
df = pd.DataFrame(deactive_helpdesk)
set_with_dataframe(ws, df)

#forums_deactive_societies
spreadsheetid = "1sqPSxZAts7fZpB5TyiJf7OY_HDkyuB79qKK0vYD4aMg"
range = "deactive_forums"
wb = gc.open_by_key(spreadsheetid)
ws = wb.worksheet(range)
df = pd.DataFrame(deactive_forums)
set_with_dataframe(ws, df)